In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd './drive/MyDrive/gaze_estimation'

In [ ]:
import time
import copy
import torch
import random
import torchvision
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm

from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from gazenet.gazenet import gazenet
from networks import *

In [ ]:
device = torch.device('cuda:0')
seed = 10

experiment_name = 'ut_3fold'
res_list = []

In [ ]:
for fold in [0,1,2] :

    train_ids = np.load(f'../ut_dataset/3-fold/fold_{fold}_train_ids.npy').reshape(-1)
    train_images = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_images.npy'), dtype=torch.float).unsqueeze(1)
    train_hps = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_2d_hps.npy'), dtype=torch.float)
    train_gazes = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_2d_gazes.npy'), dtype=torch.float)

    test_ids = np.load(f'../ut_dataset/3-fold/fold_{fold}_test_ids.npy').reshape(-1)
    test_images = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_images.npy'), dtype=torch.float).unsqueeze(1)
    test_hps = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_2d_hps.npy'), dtype=torch.float)
    test_gazes = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_2d_gazes.npy'), dtype=torch.float)

    res_list.append(gazenet(train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        VGG16_rev.VGG16, max_iter = 15000,
        device=device, experiment_name = f'{experiment_name}_{fold}', SEED = seed + fold, verbose=False, large_test=True))